# Twitter Data Analysis

This notebook demonstrates how to use the Twitter API v2 script and analyze the collected data.

In [1]:
# Import necessary libraries
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys, os
sys.path.append("../scripts")

from twitterapiv2 import get_all_tweets, default_query_parameters, request_headers
import os

## Step 1: Fetch Tweets

First, let's fetch tweets using our Twitter API script.

In [ ]:
# Set up headers with your bearer token
bearer_token = os.getenv("X_BEARER_TOKEN")
headers = request_headers(bearer_token)

In [ ]:
# Fetch tweets
tweets = get_all_tweets(
    "https://api.twitter.com/2/tweets/search/recent",
    headers,
    default_query_parameters,
    max_pages=3 # Adjust this number based on your needs
)

print(f"Fetched {len(tweets)} tweets")

## Step 2: Convert to DataFrame

Let's convert the tweets data into a pandas DataFrame for easier analysis.

In [ ]:
# Convert to DataFrame
df = pd.DataFrame(tweets)

# Display basic information
print("DataFrame Info:")
df.info()

print("\nFirst few tweets:")
df.head()

## Step 3: Basic Analysis

Let's perform some basic analysis on the tweets.

In [ ]:
# Convert created_at to datetime
df['created_at'] = pd.to_datetime(df['created_at'])

# Extract engagement metrics
df['retweet_count'] = df['public_metrics'].apply(lambda x: x['retweet_count'])
df['reply_count'] = df['public_metrics'].apply(lambda x: x['reply_count'])
df['like_count'] = df['public_metrics'].apply(lambda x: x['like_count'])
df['quote_count'] = df['public_metrics'].apply(lambda x: x['quote_count'])

# Display basic statistics
print("Engagement Statistics:")
df[['retweet_count', 'reply_count', 'like_count', 'quote_count']].describe()

## Step 4: Visualization

Let's create some visualizations to better understand the data.

In [ ]:
# Set style
plt.style.use('seaborn')

# Create a figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Tweet timeline
df.set_index('created_at').resample('D').size().plot(ax=axes[0,0], title='Tweets per Day')

# Plot 2: Engagement metrics
engagement_metrics = ['retweet_count', 'reply_count', 'like_count', 'quote_count']
df[engagement_metrics].boxplot(ax=axes[0,1])
axes[0,1].set_title('Engagement Metrics Distribution')
axes[0,1].tick_params(axis='x', rotation=45)

# Plot 3: Correlation heatmap
sns.heatmap(df[engagement_metrics].corr(), annot=True, ax=axes[1,0])
axes[1,0].set_title('Engagement Metrics Correlation')

# Plot 4: Top 10 most liked tweets
top_liked = df.nlargest(10, 'like_count')[['text', 'like_count']]
top_liked.plot(kind='barh', x='text', y='like_count', ax=axes[1,1])
axes[1,1].set_title('Top 10 Most Liked Tweets')

plt.tight_layout()
plt.show()

## Step 5: Save Processed Data

Finally, let's save our processed data for future use.

In [ ]:
# Save to CSV
df.to_csv('processed_tweets.csv', index=False)
print("Data saved to processed_tweets.csv")